## Cómo crear y usar un kernel de Jupyter a partir de un entorno virtual creado con `uv`

Sigue estos pasos para que tu entorno virtual (creado con `uv`) pueda ser seleccionado y usado como kernel en Jupyter Notebook o Jupyter Lab:

1. **Activa tu entorno virtual**
   ```bash
   source .venv/bin/activate

(Reemplaza .venv por el nombre de tu entorno si es diferente)

2. Instala ipykernel en el entorno
   ```bash
   pip install ipykernel

3. Crea y registra el kernel para Jupyter
   ```bash
   python -m ipykernel install --user --name mlops-udm --display-name "mlops udm"
   
* Pon el nombre que deseas, en este caso usaré mlops-udm pero puedes asignar el que desees. 

* Puedes cambiar mlops-udm y "mlops udm" por el nombre que prefieras para identificar tu kernel.

----> Ve al menú superior: Kernel > Cambiar kernel... y elige "mlops udm".
_______

## Escenario 1: Un cientifico de datos participando de una competición en kaggle

MLflow setup:
* Tracking server: no
* Backend store: sistema de archivos local
* Artifacts store: sistema de archivos local

Usando MLFlow Ui podemos revisar los resultados

In [1]:
import mlflow

En este caso estamos usando el path actual para definir donde queremos guardar las ejecuciones

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///Users/mdurango/University/MLOps/02-Experiment-Tracking/scenarios/mlruns'


Define la ubicación donde se almacenarán los metadatos de los experimentos. En este caso, apunta al directorio mlruns dentro de la carpeta scenarios.

In [20]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/mdurango/University/MLOps/02-Experiment-Tracking/scenarios/mlruns/0', creation_time=1758644392136, experiment_id='0', last_update_time=1758644392136, lifecycle_stage='active', name='Default', tags={}>]

### Creemos un experimento y logguemos la corrida

Hagamos una prueba con un dataset de juguete, aca definimos siempre un experimento de mlflow de la siguiente manera:

#### Definición del Experimento

```python 
mlflow.set_experiment("my-experiment-1")
```

Crea o selecciona un experimento llamado "my-experiment-1" donde se registrarán todas las ejecuciones relacionadas.

#### Ejecución del Experimento

```python
with mlflow.start_run():                    
    # Código del experimento
```

El contexto with mlflow.start_run() inicia automáticamente una nueva ejecución y la finaliza al salir del bloque.


## Flujo del Experimento

1. Carga de Datos

- Utiliza el dataset de iris de scikit-learn como ejemplo.

2. Definición de Parámetros: 
- C: Parámetro de regularización para LogisticRegression
- random_state: Semilla para reproducibilidad

- mlflow.log_params() registra estos parámetros para futura referencia

3. Entrenamiento del Modelo 

- Entrena un modelo de regresión logística con los parámetros especificados.

4. Evaluación y Logging de Métricas

- Calcula predicciones en el conjunto de entrenamiento
- Registra la métrica de precisión (accuracy) en MLflow

5. Logging del Modelo

- Guarda el modelo entrenado como un artifact, permitiendo su reutilización posterior.

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, f1_score, precision_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run(run_name="logistic_regression"):

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.3, "random_state": 42}
    mlflow.log_params(params)

    mlflow.set_tags({"developer": "Maria Durango",
                    "module": "mlops_tracking",
                    "model": "logistic_regression"})


    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))


    mlflow.sklearn.log_model(lr, name="models", input_example=X[[0]])
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/09/23 12:08:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


default artifacts URI: 'file:///Users/mdurango/University/MLOps/02-Experiment-Tracking/scenarios/mlruns/771184442123308775/9f79d39d0d574f018cc738ea90157634/artifacts'


In [31]:
# import Random Forest

from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run(run_name="random_forest"):
    # Código del experimento
    params = {"n_estimators": 100, "random_state": 42}

    X, y = load_iris(return_X_y=True)

    mlflow.log_params(params)

    rf = RandomForestClassifier(**params).fit(X, y)
    y_pred = rf.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    mlflow.set_tags({"developer": "Maria Durango",
                    "module": "mlops_tracking",
                    "model": "random_forest"})


    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")


default artifacts URI: 'file:///Users/mdurango/University/MLOps/02-Experiment-Tracking/scenarios/mlruns/771184442123308775/2a081a476a68475ea7125c1bab1becc5/artifacts'


## Exploración de Resultados

### Búsqueda de Experimentos

Lista todos los experimentos disponibles en el tracking server.


In [24]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/mdurango/University/MLOps/02-Experiment-Tracking/scenarios/mlruns/771184442123308775', creation_time=1758647158704, experiment_id='771184442123308775', last_update_time=1758647158704, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///Users/mdurango/University/MLOps/02-Experiment-Tracking/scenarios/mlruns/0', creation_time=1758644392136, experiment_id='0', last_update_time=1758644392136, lifecycle_stage='active', name='Default', tags={}>]

Filtrando por texto: 

In [25]:
mlflow.search_experiments(filter_string="name = 'my-experiment-1'")

[<Experiment: artifact_location='file:///Users/mdurango/University/MLOps/02-Experiment-Tracking/scenarios/mlruns/771184442123308775', creation_time=1758647158704, experiment_id='771184442123308775', last_update_time=1758647158704, lifecycle_stage='active', name='my-experiment-1', tags={}>]

Filtrando por ID

In [26]:
mlflow.search_runs(experiment_ids=['771184442123308775'])

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.n_estimators,params.random_state,params.C,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.runName,tags.developer,tags.mlflow.source.type,tags.model,tags.module
0,1b7eba94b8d740e083d8dc4a081887d9,771184442123308775,FINISHED,file:///Users/mdurango/University/MLOps/02-Exp...,2025-09-23 17:06:16.257000+00:00,2025-09-23 17:06:16.326000+00:00,1.000000,100,42,None,mdurango,/Users/mdurango/University/MLOps/.venv/lib/pyt...,unequaled-finch-200,Maria Durango,LOCAL,random_forest,mlops_tracking
1,ba66db332c2f479295bd5cd10bbf6b1c,771184442123308775,FAILED,file:///Users/mdurango/University/MLOps/02-Exp...,2025-09-23 17:06:02.915000+00:00,2025-09-23 17:06:03.030000+00:00,1.000000,100,42,None,mdurango,/Users/mdurango/University/MLOps/.venv/lib/pyt...,melodic-chimp-247,None,LOCAL,None,None
2,09b9f509957447d0afa3df6a3df7ecb7,771184442123308775,FINISHED,file:///Users/mdurango/University/MLOps/02-Exp...,2025-09-23 17:05:58.705000+00:00,2025-09-23 17:06:00.737000+00:00,0.966667,None,42,0.3,mdurango,/Users/mdurango/University/MLOps/.venv/lib/pyt...,polite-bass-209,Maria Durango,LOCAL,logistic_regression,mlops_tracking


In [27]:
mlflow.search_runs(experiment_ids=['771184442123308775']).sort_values(by="metrics.accuracy", ascending=False)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.n_estimators,params.random_state,params.C,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.runName,tags.developer,tags.mlflow.source.type,tags.model,tags.module
0,1b7eba94b8d740e083d8dc4a081887d9,771184442123308775,FINISHED,file:///Users/mdurango/University/MLOps/02-Exp...,2025-09-23 17:06:16.257000+00:00,2025-09-23 17:06:16.326000+00:00,1.000000,100,42,None,mdurango,/Users/mdurango/University/MLOps/.venv/lib/pyt...,unequaled-finch-200,Maria Durango,LOCAL,random_forest,mlops_tracking
1,ba66db332c2f479295bd5cd10bbf6b1c,771184442123308775,FAILED,file:///Users/mdurango/University/MLOps/02-Exp...,2025-09-23 17:06:02.915000+00:00,2025-09-23 17:06:03.030000+00:00,1.000000,100,42,None,mdurango,/Users/mdurango/University/MLOps/.venv/lib/pyt...,melodic-chimp-247,None,LOCAL,None,None
2,09b9f509957447d0afa3df6a3df7ecb7,771184442123308775,FINISHED,file:///Users/mdurango/University/MLOps/02-Exp...,2025-09-23 17:05:58.705000+00:00,2025-09-23 17:06:00.737000+00:00,0.966667,None,42,0.3,mdurango,/Users/mdurango/University/MLOps/.venv/lib/pyt...,polite-bass-209,Maria Durango,LOCAL,logistic_regression,mlops_tracking


### Interacción con el model registry

Permite interactuar programáticamente con el tracking server y model registry.

In [33]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [34]:
client.search_registered_models()

[]

In [35]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")